In [1]:
using LinearAlgebra
using JuMP
using GLPK

model = Model(GLPK.Optimizer)

@variable(model, x[1:4] >= 0) # x1, x2, x3, x4

@constraint(model, x[1] + x[3] >= 3)
@constraint(model, x[2] + 2x[3] + x[4] >= 4)
@constraint(model, x[4] >= 2)

@objective(model, Min, 3x[2] + 4x[3] + 5x[4])

optimize!(model)


println("Solution optimale :")
for i in 1:4
    println("x[$i] = ", value(x[i]))
end
println("Valeur de la fonction objectif : ", objective_value(model))

Solution optimale :
x[1] = 2.0
x[2] = 0.0
x[3] = 1.0
x[4] = 2.0
Valeur de la fonction objectif : 14.0
